In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time


We are going to read the energy eigenvalues and dipole matrix elements from .npy files.  We will also still create an instance of the PFHamiltonianGenerator class so we can use its build_pcqed_pf_hamiltonian() method, but it is not really important what details we use to instantiate this class... so we will use LiH in a minimal basis since this is a fast way to instantiate the class!

In [ ]:
# read data from .npy files for formaldehyde casci(8,8) calculations

# !!! Change this to the correct path on your computer!
npy_folder = "/Users/jfoley19/Code/data_repository/Mapol/LiH/PCQED/6-311G/r_1.4/"

# these file names should still be good
E_npy_file = npy_folder + "LiH_r_1.4_6311g_fci_Energies.npy"
Mu_npy_file = npy_folder + "LiH_r_1.4_6311g_fci_Dipoles.npy"

# store energy eigenvalues in E_array
E_array = np.load(E_npy_file)
# store dipole matrix elements in Mu_array
Mu_array = np.load(Mu_npy_file)

# print their shape so we know how many elements we have
print(np.shape(E_array))
print(np.shape(Mu_array))

# setup basic arguments to create an instance of the PFHamiltonianGenerator class
mol_str = """
Li
H 1 1.4
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}


cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}

# create the instance of our PFHamiltonianGenerator class
instance = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)

In [ ]:


N_el = 25
N_ph = 2
omega = 0.12086
lambda_vector = np.array([0., 0., 0.01])


fast_start = time.time()
instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array, neglect_DSE=True)
fast_end = time.time()

print(F"Fast build took {fast_end-fast_start} seconds")

print(instance.PCQED_pf_eigs[0])


Here we want to try to analyze the Coherent state transformation from the point of view of a Shrieffer-Wolf transformation.  

In a perturbative approach to SW transformations, we have the following unitary transformation:

$$ \hat{U} = {\rm exp}\left[ \epsilon \hat{S} \right] \tag{1} $$

that can act on the Hamiltonian that is written as 

$$ \hat{H} = \hat{H}_0 + \epsilon \hat{H}' \tag{2} $$
where $\hat{H}_0$ is diagonal and $\hat{H}'$ is off-diagonal.  

We can write the transformation as 

$$ \hat{U} \hat{H} \hat{U}^{\dagger} = \hat{H} + \left[ \hat{S}, \hat{H} \right] + \frac{1}{2} \left[ \hat{S}, \left[ \hat{S}, \hat{H} \right] \right] + ... \tag{3} $$

If we use the right hand side of Eq. (2) into the expansion and organize by orders of $\epsilon$, we get

$$ \hat{U} \hat{H} \hat{U}^{\dagger} = \hat{H}_0 +  \epsilon \left( \hat{H}' + \left[ \hat{S}, \hat{H}_0 \right]  \right) + \epsilon^2  \left( \left[ \hat{S}, \hat{H}' \right] + \frac{1}{2} \left[ \hat{S}, \left[ \hat{S}, \hat{H}_0 \right] \right] \right) + ... \tag{4} $$

One strategy is to have the perturbation vanish at 1st order, which is equivalent to having 

$$  \left[ \hat{S}, \hat{H}_0 \right] = -\hat{H}' $$

With that, we then see that the second order term becomes

\begin{align}
\left[ \hat{S}, \hat{H}' \right] + \frac{1}{2} \left[ \hat{S}, \left[ \hat{S}, \hat{H}_0 \right] \right] &= \left[ \hat{S}, \hat{H}' \right] - \frac{1}{2} \left[ \hat{S}, \hat{H}' \right] \\
&= \frac{1}{2} \left[ \hat{S}, \hat{H}' \right].
\end{align}

We then can write an effective Hamiltonian that is second order in $\epsilon$ as 

$$ \hat{H}_{eff} = \hat{H}_0 + \frac{1}{2} \left[ \hat{S}, \hat{H}' \right]. $$

The coherent state transformation can be viewed in a similar spirit with $\hat{S} = -z \left(\hat{b}^{\dagger} - \hat{b} \right)$,
where $z = \frac{\langle d \rangle}{\sqrt{2 \omega}}$.  

Let's examine the following quantities:

$\hat{H}_0 = \hat{H}_{el} + \hat{H}_{ph}$ and $\hat{H}' = \hat{H}_{blc} + \hat{H}_{dse}$:



In [ ]:
instance.fast_build_pcqed_pf_hamiltonian(2, 2, omega, lambda_vector, E_array, Mu_array, neglect_DSE=False)

print("Full PF")
print(instance.PCQED_H_PF)
print("Electronic")
print(instance.PCQED_H_EL)
print("Photonic")
print(instance.PCQED_H_PH)
print("DSE")
print(instance.PCQED_H_DSE)
print("BLC")
print(instance.PCQED_H_BLC)


In [ ]:
# S matrix as the Coherent State Transformation
z = - np.dot(lambda_vector, Mu_array[0,0,:]) / np.sqrt(2 * omega)

S = np.array([[0, 0, z, 0],
             [0, 0, 0, z],
             [-z, 0, 0, 0],
             [0, -z, 0, 0]]
             )

print(S)

# Notes on the matrix blocks

\begin{align}\label{EQN:projected_matrix}
{\bf \mathcal{H}} =
&\begin{bmatrix}
{\bf E} + {\bf D}   & -\sqrt{\frac{\omega}{2}} {\bf d}  & 0 & \dots & 0 & 0 \\
-\sqrt{\frac{\omega}{2}}{\bf d} & {\bf E} + {\bf D} + {\bf \Omega}   & -\sqrt{\omega} {\bf d}  & \dots & 0 & 0 \\
0   &  -\sqrt{\omega}{\bf d} & {\bf E} + {\bf D} + 2{\bf \Omega} & \dots & 0  & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0      &      0 &   0    &  \dots     & {\bf E} + {\bf D} + (N-1){\bf \Omega}    & -\sqrt{\frac{N \omega}{2}} {\bf d} \\
0      &      0 &   0    &  \dots     & -\sqrt{\frac{N \omega}{2}}{\bf d}     & {\bf E} + {\bf D} + N{\bf \Omega}. 
\end{bmatrix}
\end{align}


Note that there are basically 4 different matrices with shape $(N_{el}, N_{el})$ that appear in this Hamiltonian matrix.  The following outlines the equations for the elements of each matrix along with one or more strategies to assemble them.  

### E matrix
#### Math
$$ E_{\alpha \beta} = \langle \psi_{\alpha} | \mathcal{H}_{el} | \psi_{\beta} \rangle = E_{\beta} \delta_{\alpha \beta} $$ 
where $E_{\beta}$ denote the electronic energy eigenvalues of the molecular system.
#### Code
If the energy eigenvalues are stored in an array called `E_array`, then we can build ${\bf E}$ by multiplying these values by an $(N_{el}, N_{el})$ identity matrix $\mathbb{I}$.

##### build N_el x N_el identity matrix
`_I = np.eye(N_el)`

##### build N_el x N_el _E matrix
`_E = E_array * _I`




In [ ]:
# code to build E matrix goes here

### $\Omega$ matrix
#### Math
$$ \Omega_{\alpha \beta} = \langle \psi_{\alpha} | \omega | \psi_{\beta} \rangle  = \omega \delta_{\alpha \beta}$$ 

#### Code
If the photon frequency is stored in the variable `omega`, then the $(N_{el},N_{el})$ matrix `_O` can be build 
by multiplying the $\mathbb{I}$ by $omega$:

##### build N_el x N_el _O matrix
`_O = omega * _I`


In [ ]:
# code to build Omega matrix goes here

### d matrix
#### Math
$$ d_{\alpha \beta} = \lambda \cdot  \langle \psi_{\alpha} | \mathcal{\mu} | \psi_{\beta} \rangle = \lambda_x \mathcal{\mu}_{x, \alpha \beta} +  \lambda_y \mathcal{\mu}_{y, \alpha \beta} +  \lambda_z \mathcal{\mu}_{z, \alpha \beta} $$ 
where $\mathcal{\mu}_{x, \alpha \beta}$ denotes the x-component of the (transition) dipole moment between molecular electronic state $\psi_{\alpha}$ and $\psi_{\beta}$.
#### Code
If the dipole matrix elements are stored in a $(N_{el}, N_{el}, 3)$  array called `mu_array` and the $\lambda$ vector is stored in an 3-element array called `\lambda_vector`, then we can build the
$(N_{el}, N_{el})$ array ${\bf d}$ by [contraction](https://en.wikipedia.org/wiki/Tensor_contraction) using `np.einsum()`:

##### build N_el x N_el _d matrix
`_d = np.einsum("k,ijk->ij", lambda_vector, mu_array)`

In [ ]:
# code to build the d matrix goes here

### D matrix
#### Math
$$ D_{\alpha \beta} = \frac{1}{2} \sum_{\gamma} d_{\alpha \gamma} d_{\gamma \beta} $$ 

#### Code
We have the $(N_{el}, N_{el})$ elements of the ${\bf d}$ array stored in `_d`.  We can then build `_D`
using matrix-matrix multiplication as follows:

`_D = _d @ _d`

or using einsum as follows:

`_D = np.einsum("ik,kj->ij",_d, _d)`
